In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


#from pyspark.context import SparkContext
#from pyspark.sql import HiveContext, SparkSession

# Creating a SparkContext is a must
#sc = SparkContext(appName="profiler")
# Optional creation of a HiveContext
#sql_context = HiveContext(sc)
# Optional creation of a SparkSession
#spark = SparkSession(sc)
#spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


#from pyspark import SparkConf, SparkContext
#from pyspark.sql import SparkSession
#from pyspark.sql.functions import *
#import subprocess
#import sys

#appname='aa'
#spark = (SparkSession.builder.appName(appname)
#         .config("hive.metastore.uris", "thrift://anp-r09mstr.c03.hadoop.td.com:9083")
#         .config("hive.metastore.client.socket.timeout", "300")
#         .config("hive.metastore.warehouse.dir", "/user/hive/warehouse")
#         .config("hive.warehouse.subdir.inherit.perms", "true")
#         .config("hive.execution.engine", "spark")
#         .config("hive.metastore.execute.setugi", "true")
#         .config("hive.support.concurrency", "true")
#         .config("hive.zookeeper.quorum",
#                 "anp-r04mstr.c03.hadoop.td.com,anp-r02mstr.c03.hadoop.td.com,anp-r01mstr.c03.hadoop.td.com")
#         .config("hive.zookeeper.client.port", "2181")
#         .config("hive.zookeeper.namespace", "hive_zookeeper_namespace_hiveHA")
#         .config("hive.cluster.delegation.token.store.class", "org.apache.hadoop.hive.thrift.DBTokenStore")
#         .config("hive.server2.enable.doAs", "false")
#         .config("hive.metastore.sasl.enabled", "true")
#         .config("hive.server2.authentication", "kerberos")
#         .config("hive.metastore.kerberos.principal", "hive/_HOST@C03.HADOOP.TD.COM")
#         .config("hive.server2.authentication.kerberos.principal",
#                 "hive/hiveserver.c03.hadoop.td.com@C03.HADOOP.TD.COM")
#         .config("hive.server2.use.SSL", "true")
#         .config("hive.jobexec.dynamic.partition", "true")
#         .config("spark.sql.crossJoin.enabled", "true")
#         .config("hive.jobexec.dynamic.partition.mode", "nonstrict").enableHiveSupport().getOrCreate())
#spark.sparkContext.setLogLevel('INFO')

#var1=str(sys.argv[1])
#var2=str(sys.argv[2])

In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F

In [ ]:
# For Joe
## read csv
datafile=spark.read.csv("/tenantspace/ida/anp/cabbtdct1/hivewarehouse/sharb24/CLCP_Dec.csv",header=True)
#datafile.show(5)

## overwrite
datafile.write.mode("overwrite").format("hive").saveAsTable("anp_cabbtdct1_sandbox.CLCP_S680")

In [ ]:
datafile.write.mode("overwrite").format("hive").saveAsTable("anp_cabbtdct1_sandbox.CUSTOMER_DECISION_JAN2023")

In [ ]:
## read SM csv
sm=spark.read.csv("/tenantspace/ida/anp/cabbtdct1/hivewarehouse/thural2/SM_HISTORY.csv",header=True)
sm.show(5)

## overwrite
sm.write.mode("overwrite").format("hive").saveAsTable("anp_cabbtdct1_working.SM_FY22_SCORECARD_HISTORY")

df=spark.sql ("select * from anp_cabbtdct1_working.SM_FY22_SCORECARD_HISTORY limit 10")
pdf=df.toPandas()
pdf.head(10)


In [ ]:
## read AM csv
am=spark.read.csv("/tenantspace/ida/anp/cabbtdct1/hivewarehouse/thural2/AM_HISTORY.csv",header=True)
am.show(5)

## overwrite
am.write.mode("overwrite").format("hive").saveAsTable("anp_cabbtdct1_working.AM_FY22_SCORECARD_HISTORY")

df=spark.sql ("select * from anp_cabbtdct1_working.AM_FY22_SCORECARD_HISTORY limit 10")
pdf=df.toPandas()
pdf.head(10)


In [ ]:
## import LEI csv file (change the tenantspace path name)
leif=spark.read.csv("/tenantspace/ida/anp/cabbtdct1/hivewarehouse/sharb24/LEI_Modified_Dec2023.csv",header=True)
leif.show(5)

## overwrite
#leif.write.mode("overwrite").format("hive").saveAsTable("anp_cabbtdct1_working.SBB_LEI")
leif.write.mode("append").format("hive").saveAsTable("anp_cabbtdct1_working.SBB_LEI_F24")


In [ ]:
temp = leif.toPandas()
temp.shape

In [ ]:
## import GNB file (change tenantspace path for username)
gnbf=spark.read.csv("/tenantspace/ida/anp/cabbtdct1/hivewarehouse/sharb24/GNB_Dec2023.csv",header=True)

## overwrite
#gnbf.write.mode("overwrite").format("hive").saveAsTable("anp_cabbtdct1_working.SBB_GNB_F24")
gnbf.write.mode("append").format("hive").saveAsTable("anp_cabbtdct1_working.SBB_GNB_F24")
#pdf=gnbf.toPandas()
#pdf.head(10)


In [ ]:
## import TDEF file --not partitioned because it is a cumulative file
tdef=spark.read.csv("/tenantspace/ida/anp/cabbtdct1/hivewarehouse/sharb24/TDEF.csv",header=True)
#tdef.show(5)

## overwrite
tdef.write.mode("overwrite").format("hive").saveAsTable("anp_cabbtdct1_working.SBB_TDEF")
pdf=tdef.toPandas()
pdf.head(10)


In [ ]:
df1=spark.sql("select distinct a.* from anp_cabbtdct1_working.SBB_LEI a inner join (select max(record_date) as max_lei from anp_cabbtdct1_working.SBB_LEI)b on a.record_date=b.max_lei")

In [ ]:
df_lei=df1.toPandas()
df_lei.head(20)

In [ ]:
df1 = spark.table('anp_cabbtdct1_working.SBB_LEI')
df2=sqlContext.sql("select max(record_date) as max_lei from anp_cabbtdct1_working.SBB_LEI")
df2 = spark.table('db.table2')
df3 = df1.join(df2, how='inner',on = F.col('record_date') == F.col('max_lei'))
df4 = df1.join(df2, how='inner',on='field_name').join(df3,how='left',on='field_name')

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df3.toPandas()

In [ ]:
df8=spark.sql("select * from bbir_vw.hrm_employee_profile limit 10")

In [ ]:
## read FY GOALS csv
goal=spark.read.csv("/tenantspace/ida/anp/cabbtdct1/hivewarehouse/sharb24/SBB_FY24_GOALS.csv",header=True)
goal.show(5)

## overwrite
goal.write.mode("overwrite").format("hive").saveAsTable("anp_cabbtdct1_working.SBB_FY_GOALS")

#df=spark.sql ("select * from anp_cabbtdct1_working.SBB_FY_GOALS limit 10")
#pdf=df.toPandas()
#pdf.head(10)

In [ ]:
df=spark.sql ("select * from anp_cabbtdct1_working.SBB_FY_GOALS")
pdf=df.toPandas()
pdf.columns.dtypes

In [ ]:
pdf.info()